# Workflows

As noted in the article ["Building effective agents" by Anthropic](https://www.anthropic.com/research/building-effective-agents), workflows can be an effective way to use LLMs to complete complex tasks. Our experience building the first iteration of the "note formatting" feature, small LLMs are not capable of completing such complicated task in one step yet. Our goal in this exploration is to see if breaking down the task in multiple steps of one workflow can significantly improve realability of the system and accuracy.

### Design

1. Original note
2. (Assess note quality)
3. Make concise (input: original note + edited note)
4. Fix code blocks (input: original note + edited note) 
5. Fix math blocks (input: original note + edited note)
6. Fix media (input: original note + edited note)
7. Fix HTML formatting (input: original note + edited note)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import html

from anki_ai.domain.model import Deck, Note
from anki_ai.adapters.chat_completion import get_completion
from anki_ai.service_layer.services import get_chat_completion, ChatCompletionsService, strip_html_tags

In [3]:
deck = Deck("default")
deck.read_txt("../data/Selected Notes v8.txt", exclude_tags=["personal"])
print(f"The deck contains {len(deck)} notes.")

The deck contains 2854 notes.


Let's take a random sample of notes, to iterate more rapidly in our initial exploration. 

In [4]:
deck.shuffle()
sample = deck[:50] # NOTE: this returs a list, should it instead return a Deck?

Now that we have a small dataset we can use, let's start working on creating those steps in the workflow. 

In [5]:
def escape(s: str) -> str:
    "Add HTML tags"
    return html.escape(s).replace('\n', '<br>')

def unescape(s: str) -> str:
    "Remove HTML tags"
    return html.unescape(s).replace("<br>", "\n")

In [6]:
chat = get_completion()

In [7]:
def apply_transform(
    system_msg: str,
    user_msg: str,
    note, 
    edited_note=None,
    chat: ChatCompletionsService = chat,
    json_mode: bool = False,
):
    messages = system_msg + "\n\n" + user_msg(note, edited_note)
    # print(messages)

    if json_mode:
        extra_body = {
            "guided_json": Note.model_json_schema(),
            "guided_whitespace_pattern": r"[\n\t ]*",
        }
    else:
        extra_body = {}

    chat_response = chat.create(
        model="meta-llama/Meta-Llama-3.1-8B-Instruct",
        prompt=messages,
        temperature=0,
        max_tokens=500,
        extra_body=extra_body,
    )

    print("\n#######################\n")
    # NOTE: send text without HTML specical characters to the LLM
    print(f"Front: {unescape(note.front)}\nBack: {unescape(note.back)}\nTags: {note.tags}\n")
    if json_mode:
        json_data = chat_response.choices[0].text
        # print(json_data)
        # print()
        new_note = Note.model_validate_json(json_data)
        front = new_note.front.replace("<br>", "\n")
        back = new_note.back.replace("<br>", "\n")
        tags = new_note.tags
        print(f"Front: {front}\nBack: {back}\nTags: {tags}")
    else:
        print(chat_response.choices[0].message.content)

    return (note, new_note)

In [8]:
system_msg = r"""Optimize this Anki note:
- Concise, simple, distinct
- Follow format rules
- Markdown syntax

Reply in this format:
Front: [edited front]
Back: [edited back]
Tags: [tags]

Terminal commands:
```bash
$ command <placeholder>
```

Code:
```language
code here
```

Write utilities and CLI tools names within inline code block: e.g., `nvim`, `systemctl`

Write keyboard keys and keymaps within inline code blocks: e.g., `<C-aa>`, `x`

Use the following placeholders only: <file>, <path>, <link>, <command>.

Represent newlines with the `<br>` tag. 

Leave code blocks and images on the original note unchanged. 

No explanations.

Return results using this JSON schema:
{
    "title": "Note",
    "type": "object",
    "properties": {
        "front": {"type": "string"},
        "back": {"type": "string"},
        "tags": {"type": "string"},
    },
    "required": ["front", "back", "tags"]
}

Example 1:
Front: What command does extract files from a zip archive?
Back: ```bash
$ unzip <file>
Tags: ['linux']
```
{ "front": "Extract zip files", "back": "\"```bash<br>$ unzip <file><br>```\"", "tags": ['linux'] }

Example 2:
Front: What is the command to print manual or get help for a command?
Back: ```bash
$ man ...
Tags: ['linux']
```
{ "front": "Get command manual/help", "back": "\"```bash<br>$ man <command><br>```\"", "tags": ['linux'] }

Example 3: 
Front: What command creates a soft link?
Back: ```bash
$ ln -s <file_name> <link_name>
```
Tags: ['linux']
{ "front": "Create soft link", "back": "\"```bash<br>$ ln -s <file> <link><br>```\"", "tags": ['linux'] }

Example 4:
Front: In the `ln -s` command, what is the order of file name and link name?
Back: ```bash
$ ln -s <file_name> <link_name>
```
Tags: ['linux']
{ "front": "`ln -s` argument order", "back": "\"<file> then <link>\"", "tags": ['linux'] }

Example 5:
Front: What is the range of the Leaky ReLU function?
Back: $ [ -0.01, + \infty ] $
Tags: ['dl']
{ "front": "Leaky ReLU range", "back": "\"$ [-0.01, +\infty] $\"", "tags": ['dl'] }

Example 6:
Front: What key returns the `^` in the shifted state?
Back: "`6`"
Tags: ['keyboard']
{ "front": "Keyboard key for `^` in shifted state", "back": "\"`6`\"", "tags": ['keyboard'] }

Example 7:
"""

user_msg = lambda note, _: f"""Front: {unescape(note.front)}\nBack: {unescape(note.back)}\nTags: {note.tags}\n"""

def format_notes(notes, chat) -> (Note, Note):
    return [apply_transform(system_msg, user_msg, note, chat, json_mode=True) for note in notes]

notes_bundle = format_notes(sample, chat)


#######################

Front: Motion to move the cursor [count] WORDS forward
Back: "`[count]W`

<img src=""HYaJn.gif"">"
Tags: ['nvim']

Front: Nvim motion WORDS forward
Back: "`[count]W`
<img src="HYaJn.gif"">"
Tags: ['nvim']

#######################

Front: "What type of tyres can a hooked rim fit?

<img src=""hooked-rim-fast-fwd-1.jpeg"">"
Back: Clincher and (if specified) tubeless-ready tyres
Tags: ['cycling']

Front: Hooked rim tyre type
Back: Clincher and (if specified) tubeless-ready tyres
Tags: ['cycling']

#######################

Front: What command line tool controls the Bluetooth  connections?
Back: `bluetoothctl`
Tags: ['linux']

Front: Bluetooth connection control
Back: "`bluetoothctl`"
Tags: ['linux']

#######################

Front: What key returns the `^` in the shifted state?
Back: "`6`

<img src=""paste-35252f02ada229d38491a63d20fa2ea9e8a5f0f8.jpg"">"
Tags: ['keyboard']

Front: Keyboard key for `^` in shifted state
Back: "`6`
<img src="paste-35252f02ada229d38491

We are making the LLM process the strings without HTML special characters and tags. The reason for doing this is that we are removing some of the complexity, and making human eval easier.

Another exception is that we make the LLM generate newlines using the HTML tag `<br>`, because we notice it confuses less the model. When using `\n`, the LLM sometimes starts generating random sequences of symbols.